In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import gradio as gr
import json
from difflib import SequenceMatcher

# Datos para el lenguaje español
nltk.download('punkt')
nltk.download('stopwords')

# Tokenización
tokenizer = Tokenizer()

# Preprocesamiento de texto
def preprocesar_texto(texto):
    tokens = word_tokenize(texto, language='spanish')
    palabras_stop = set(stopwords.words('spanish'))
    palabras_stop.add('.')
    palabras_claves = [word.lower() for word in tokens if word.lower() not in palabras_stop]
    return palabras_claves

# Nuevos textos y etiquetas para entrenamiento
texts = [
    "Mañana juega Portugal vs Turquia a las 11 de la mañana por ESPN. Todos con el Bicho SIUUUUU",
    "La película de los Vengadores es impresionante, me encanta Iron Man y Thor",
    "Amo la trilogía de El Señor de los Anillos, especialmente La Comunidad del Anillo",
    "Mi parte favorita de Harry Potter es la batalla en Hogwarts en Las Reliquias de la Muerte",
    "El Padrino es una obra maestra del cine, la actuación de Marlon Brando es icónica",
    "La saga de Star Wars es una de las más populares, especialmente la trilogía original.",
    "Breaking Bad es una serie impresionante sobre la transformación de un profesor de química en un narcotraficante.",
    "El Rey León es una película animada clásica que encanta a todas las generaciones.",
    "Titanic cuenta la trágica historia del famoso barco y un amor imposible.",
    "Matrix es una película revolucionaria sobre una realidad simulada y un elegido que la liberará.",
    "Inception es un thriller de ciencia ficción sobre los sueños dentro de sueños.",
    "Gladiator cuenta la historia de un general romano convertido en gladiador en busca de venganza.",
    "Jurassic Park es una película icónica sobre un parque de dinosaurios que se vuelve incontrolable.",
    "The Godfather II sigue la saga de la familia Corleone, explorando los orígenes y la expansión del imperio criminal.",
    "Forrest Gump es una historia conmovedora sobre un hombre con una capacidad de comprender el mundo de manera única.",
    "La La Land es un musical moderno que cuenta una historia de amor y sueños en Los Ángeles.",
    "Avengers: Endgame es la culminación épica de la saga del infinito del Universo Cinematográfico de Marvel.",
    "Parasite es una película surcoreana que mezcla comedia negra y thriller social de forma brillante.",
    "Coco es una animación de Pixar que celebra la cultura mexicana y la importancia de la familia.",
    "La Casa de Papel es una serie española sobre un grupo de criminales que lleva a cabo el mayor atraco de la historia.",
    "Stranger Things es una serie de ciencia ficción y horror que rinde homenaje a los años 80.",
    "Black Panther es una película de superhéroes que celebra la cultura africana y la diversidad.",
    "Toy Story es una saga animada que sigue las aventuras de juguetes que cobran vida.",
    "Interstellar es una película de ciencia ficción sobre un grupo de astronautas que viajan a través de un agujero de gusano.",
    "The Dark Knight es una película de superhéroes que redefine el género con una narrativa oscura y compleja."
]

labels = [i for i in range(len(texts))]

# Ajustar el tokenizer con los nuevos textos
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=20)

# Convertir etiquetas a array numpy
labels = np.array(labels)

# Dividir los datos en entrenamiento y prueba manualmente
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=20),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Guardar el modelo
model.save('modelo_peliculas.h5')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Epoch 1/50
1/1 [==============================] - 11s 11s/step - loss: 3.2196 - accuracy: 0.0000e+00 - val_loss: 3.2264 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 137ms/step - loss: 3.2169 - accuracy: 0.0625 - val_loss: 3.2325 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 126ms/step - loss: 3.2073 - accuracy: 0.0625 - val_loss: 3.2408 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 125ms/step - loss: 3.1991 - accuracy: 0.1250 - val_loss: 3.2500 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 129ms/step - loss: 3.1946 - accuracy: 0.0625 - val_loss: 3.2603 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 122ms/step - loss: 3.2057 - accuracy: 0.0000e+00 - val_loss: 3.2686 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 127ms/step - loss: 3.1946 - accuracy: 0.0625 - val_loss: 3.2779 - val_accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Función para calcular la similitud entre dos cadenas
def calcular_similitud(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Cargar la base de datos de películas desde el archivo JSON
with open('movies.json', 'r') as f:
    data = json.load(f)

# Función para encontrar las 5 películas más similares y formatear la salida
def encontrar_peliculas_similares(descripcion):
    try:
        palabras_claves_usuario = preprocesar_texto(descripcion)
        if not palabras_claves_usuario:
            return "No se encontraron palabras clave en la descripción proporcionada."

        peliculas_similares = []

        for pelicula in data["movies"]:
            palabras_claves_pelicula = pelicula["metadata"][0]["Value"]
            palabras_claves_pelicula = [word.lower() for word in palabras_claves_pelicula]  # Convertir a minúsculas
            puntaje_similitud = 0

            for palabra_usuario in palabras_claves_usuario:
                similitudes = [calcular_similitud(palabra_usuario, palabra_pelicula) for palabra_pelicula in palabras_claves_pelicula]
                puntaje_similitud += max(similitudes, default=0)  # Tomar la mayor similitud para cada palabra del usuario

            if puntaje_similitud > 0:
                peliculas_similares.append((pelicula["nombre"], puntaje_similitud, pelicula["genero"], pelicula["calificacion"]))

        peliculas_similares.sort(key=lambda x: x[1], reverse=True)
        peliculas_similares_filtradas = peliculas_similares[:3]  # Tomar las 5 películas más similares

        if peliculas_similares_filtradas:
            resultados = []
            for pelicula in peliculas_similares_filtradas:
                nombre = pelicula[0]
                genero = ", ".join(pelicula[2])
                calificacion = pelicula[3]
                resultados.append(f"Título: {nombre}\nGénero: {genero}\nCalificación: {calificacion}")
            return "\n\n".join(resultados)
        else:
            return "No se encontraron películas que coincidan con la descripción."

    except Exception as e:
        return f"Se produjo un error: {str(e)}"

# Crear la interfaz de Gradio
iface = gr.Interface(
    fn=encontrar_peliculas_similares,
    inputs="text",
    outputs="text",
    title="MOVIE AI",
    description="¿Qué Película buscas?"
)

# Iniciar la interfaz
iface.launch()
#encontrar_peliculas_similares(str(input("Ingresa una película: ")))

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6e7cd6c030f73961b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Bases de datos
import pandas as pd
import json

# Cargar los datos JSON
data = {
    "movies": [
        {
            "nombre": "Oppenheimer",
            "ano": 2023,
            "descripcion": "La historia del científico estadounidense J. Robert Oppenheimer y su papel en el desarrollo de la bomba atómica.",
            "genero": ["Biografia", "Drama", "Historia"],
            "calificacion": 8.3,
            "duracion": 180,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["bomba atómica", "Segunda Guerra Mundial", "científico", "proyecto Manhattan", "física", "J. Robert Oppenheimer", "historia estadounidense", "Newton", ""]
                }
            ]
        },
        {
            "nombre": "Deadpool",
            "ano": 2016,
            "descripcion": "Luego de que experimentaran con él, un mercenario se vuelve inmortal pero feo, y se propone rastrear al hombre que arruinó su apariencia.",
            "genero": ["Accion", "Comedia"],
            "calificacion": 8.0,
            "duracion": 108,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Anti-héroe", "Mercenario", "Humor negro", "Mutante", "Venganza", "Violento", "Irreverente"]
                }
            ]
        },
        {
            "nombre": "Duna",
            "ano": 2021,
            "descripcion": "Adaptación de la novela de ciencia ficción de Frank Herbert, sobre el hijo de una familia noble encargada de proteger un planeta especiado.",
            "genero": ["Accion", "Aventura","Drama"],
            "calificacion": 8.0,
            "duracion": 155,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Sci-fi", "Frank Herbert", "Planeta espacial", "Nobleza", "Aventura épica", "Intriga política", "Guerras interplanetarias", "Cultura futurista", "Profecía", "Resistencia"]
                }
            ]
        },
        {
            "nombre": "Godzilla vs. Kong",
            "ano": 2021,
            "descripcion": "Cuando el gigante Kong se enfrenta al imparable Godzilla, el mundo espera cual de ellos se convertirá en el Rey de los monstros.",
            "genero": ["Accion", "Ciencia Ficcion","Suspenso"],
            "calificacion": 6.3,
            "duracion": 113,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Monstruos", "Enfrentamiento épico", "Acción", "Gigantes", "Rey de los monstruos", "Kaiju", "Batalla", "Destrucción", "Apocalipsis", "Suspenso"]
                }
            ]
        },
        {
            "nombre": "Godzilla",
            "ano": 2014,
            "descripcion": "La Tierra está en peligro por la aparición de criaturas monstruosas, pero una de ella podría ser la única que salve a la humanidad.",
            "genero": ["Accion", "Ciencia Ficcion","Aventura"],
            "calificacion": 6.4,
            "duracion": 123,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Monstruos", "Amenaza global", "Acción", "Criaturas gigantes", "Salvacion", "Apocalipsis", "Batalla épica", "Rescate", "Drama", "Suspenso"]
                }
            ]
        },
        {
            "nombre": "Kung Fu Panda 3",
            "ano": 2016,
            "descripcion": "Po se enfrenta a dos amenazas de dimensiones épicas: una sobrenatural, y otra más cotidiana.",
            "genero": ["Accion", "Animacion","Aventura"],
            "calificacion": 7.1,
            "duracion": 95,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Kung Fu", "Panda", "Aventura épica", "Animación", "Acción", "Humor", "Amenazas", "Aprendizaje", "Familia", "Amistad"]
                }
            ]
        },
        {
            "nombre": "It (Eso)",
            "ano": 2017,
            "descripcion": "En el verano de 1989 un grupo de niños deciden unirse para destruir a un monstruo que cambia de forma y se disfraza de payaso atacando los niños de Derry, un pequeño pueblo de Maine.",
            "genero": ["Fantasma", "Terror"],
            "calificacion": 7.3,
            "duracion": 135,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Globo Rojo", "Eso", "payaso", "monstruo", "niños", "1989", "Derry", "Maine", "terror", "miedo", "fantasma", "amistad", "infancia", "pueblo", "adaptación", "Stephen King", "pesadillas", "aterrorizar", "creepy"]
                }
            ]
        },
        {
            "nombre": "Furiosa",
            "ano": 2024,
            "descripcion": "La historia de origen de la guerrera renegada Furiosa antes de formar equipo con Mad Max en 'Fury Road.",
            "genero": ["Accion", "Ciencia Ficcion","Aventura"],
            "calificacion": 7.7,
            "duracion": 148,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Furiosa", "Mad Max", "guerrera", "renegada", "origen", "acción", "ciencia ficción", "aventura", "post-apocalíptico", "road movie", "lucha", "desierto", "batalla", "poder", "supervivencia", "vehículos", "motorizado", "furia", "heroína"]
                }
            ]
        },
        {
            "nombre": "Hasta el último hombre",
            "ano": 2016,
            "descripcion": "El médico del ejército estadounidense Desmond T. Doss, se niega a matar personas y se convierte en el primer hombre en la historia de Estados Unidos en recibir la Medalla de Honor sin disparar un tiro.",
            "genero": ["Accion", "Ciencia Ficcion","Aventura"],
            "calificacion": 8.1,
            "duracion": 139,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Desmond T. Doss", "Medalla de Honor", "ejército estadounidense", "objeción de conciencia", "valentía", "heroísmo", "guerra", "acción", "aventura", "historia real"]
                }
            ]
        },
        {
            "nombre": "Megalodon",
            "ano": 2018,
            "descripcion": "Un barco militar en busca de un sumergible no identificado se encuentra cara a cara con un tiburón gigante, obligado a utilizar sólo lo que tiene a bordo para defenderse de la monstruosa bestia.",
            "genero": ["Accion", "Suspenso"],
            "calificacion": 2.8,
            "duracion": 114,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["tiburón gigante", "barco militar", "sumergible", "acción", "suspenso", "defensa", "monstruosa bestia", "enfrentamiento", "mar", "aventura"]
                }
            ]
        },
        {
            "nombre": "Wolverine: Inmortal",
            "ano": 2013,
            "descripcion": "Wolverine viene a Japón para encontrarse con un viejo amigo cuya vida salvó hace años, y se ve envuelto en una conspiración que involucra a mutantes.",
            "genero": ["Accion", "Ciencia Ficcion"],
            "calificacion": 6.7,
            "duracion": 126,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["mutantes", "conspiración", "Japón", "acción", "ciencia ficción", "amistad", "poderes", "aventura", "lucha", "salvación"]
                }
            ]
        },
        {
            "nombre": "Intensa Mente 2",
            "ano": 2024,
            "descripcion": "Riley, en su adolescencia, al encuentro de nuevas emociones.",
            "genero": ["Animacion", "Aventura", "Comedia"],
            "calificacion": 7.9,
            "duracion": 96,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["adolescencia", "emociones", "aventura", "ansiedad", "crecimiento", "cambio", "familia", "amistad", "desafíos", "animación"]
                }
            ]
        },
        {
            "nombre": "Intensa Mente",
            "ano": 2015,
            "descripcion": "Riley es una chica que padece toda clase de sentimientos, y aunque su vida ha estado marcada por la alegría, también se ve afectada por otras emociones. Una serie de acontecimientos hacen que Alegría y Tristeza se mezclen en una peligrosa aventura que dará un vuelco al mundo de Riley.",
            "genero": ["Animacion", "Aventura", "Comedia"],
            "calificacion": 8.1,
            "duracion": 95,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["emociones", "aventura", "alegría", "tristeza", "desafíos", "cambio", "adolescencia", "animación", "familia", "amistad"]
                }
            ]
        },
        {
            "nombre": "Un lugar en silencio",
            "ano": 2018,
            "descripcion": "En un mundo post apocalíptico una familia es forzada a vivir en silencio mientras se esconde de los monstruos con un oído ultra sensible.",
            "genero": ["Drama", "Terror", "Ciencia Ficcion"],
            "calificacion": 7.5,
            "duracion": 90,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["apocalipsis", "monstruos", "supervivencia", "familia", "silencio", "terror", "ciencia ficción", "suspense", "post-apocalíptico", "oído ultra sensible"]
                }
            ]
        },
        {
            "nombre": "Bad Boys Hasta La Muerte",
            "ano": 2024,
            "descripcion": "Cuarta entrega de la saga de películas 'Bad Boys'.",
            "genero": ["Accion", "Aventura", "Comedia"],
            "calificacion": 7.0,
            "duracion": 115,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Bad Boys 4", "corrupción policial", "mafia mexicana", "fugitivos", "investigación", "acción", "comedia de acción", "buddy cop", "investigación criminal", "limpieza de nombre"]
                }
            ]
        },
        {
            "nombre": "El Planeta De Los Simios: Nuevo Reino",
            "ano": 2024,
            "descripcion": "Muchos años después del reinado de César, un joven simio emprende un viaje que le llevará a cuestionar todo lo que le han enseñado sobre el pasado y a tomar decisiones que definirán un futuro para simios y humanos por igual.",
            "genero": ["Accion", "Aventura", "Drama"],
            "calificacion": 7.2,
            "duracion": 145,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Simios", "Reinado", "Joven", "Viaje", "Cuestionamiento", "Pasado", "Decisiones", "Futuro", "Accion", "Aventura", "Drama"]
                }
            ]
        },
        {
            "nombre": "El planeta de los simios: la guerra",
            "ano": 2017,
            "descripcion": "Las pérdidas de los Simios son incontables. César lucha contra sus instintos más oscuros y se embarca en una misión mística para vengar a su especie.",
            "genero": ["Accion", "Aventura", "Drama"],
            "calificacion": 7.4,
            "duracion": 160,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Simios", "César", "Guerra", "Accion", "Aventura", "Drama", "Venganza", "Instintos", "Humanos vs monos", "Monos Inteligentes"]
                }
            ]
        },
        {
            "nombre": "Transformers: El Despertar De Las Bestias",
            "ano": 2023,
            "descripcion": "Sigue nuevamente a un grupo de Decepticons y Autobots que se enfrentan en una guerra entre aquellos que desean controlar el planeta y los que quieren vivir en comunión.",
            "genero": ["Accion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 6.0,
            "duracion": 127,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Transformers", "Decepticons", "Autobots", "Animales Robots", "Planeta", "Accion", "Aventura", "Ciencia Ficcion", "Robots"]
                }
            ]
        },
        {
            "nombre": "Transformers: el último caballero",
            "ano": 2017,
            "descripcion": "Una amenaza reaparece en la tierra y la casería por un artefacto perdido desata la guerra entre los Autobots y los Decepticons.",
            "genero": ["Accion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 5.2,
            "duracion": 154,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                     "Value": ["Transformers", "Autobots", "Decepticons", "Tierra", "Artefacto", "Guerra", "Autos que se transforman", "Aventura", "Peleas de robots", "Robots"]
                }
            ]
        },
        {
            "nombre": "Batman: El caballero de la noche asciende",
            "ano": 2012,
            "descripcion": "Ocho años después de la anarquía del Joker, Batman, con la ayuda de la enigmática Catwoman, se ve obligado a abandonar su exilio para salvar Gotham City, ahora al borde de la aniquilación total causada por el brutal guerrillero terrorista Bane.",
            "genero": ["Accion", "Drama", "Suspenso"],
            "calificacion": 8.4,
            "duracion": 164,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Batman", "Gotham City", "Catwoman", "Joker", "Anarquía", "Exilio", "Salvación", "Bane", "Guerrilla", "Murcielago", "Niño Huerfano"]
                }
            ]
        },
        {
            "nombre": "Avengers: Endgame",
            "ano": 2019,
            "descripcion": "Tras los devastadores sucesos de Avengers: Infinity War, el universo está en ruinas pero de los que quedan, los Avengers se unirán una vez más para deshacer la destrucción de Thanos y restaurar el orden del mundo.",
            "genero": ["Accion", "Drama", "Aventura"],
            "calificacion": 8.4,
            "duracion": 181,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Thanos", "Avengers", "Destrucción", "Restauración", "Universo", "Heroísmo", "Reunión", "Batalla final", "Viaje en el tiempo", "Thor gordo", "Capitana Marvel", "Espacio"]
                }
            ]
        },
        {
            "nombre": "Avengers: Infinity War",
            "ano": 2018,
            "descripcion": "Los Avengers y sus aliados deberán estar dispuestos a sacrificarlo todo con el fin de derrotar a Thanos antes de que él ponga fin al universo.",
            "genero": ["Accion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 8.4,
            "duracion": 149,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Thanos", "Avengers", "Aliados", "Universo", "Sacrificio", "Batalla épica", "Gemas del infinito", "Destrucción", "Heroísmo", "Le tiran una luna a iroman", "Espacio"]
                }
            ]
        },
        {
            "nombre": "Spider-Man: Sin Camino A Casa",
            "ano": 2021,
            "descripcion": "Con la identidad de Spider-Man revelada, Peter le pide ayuda al Doctor Strange. Cuando un hechizo sale mal, comienzan a aparecer enemigos de otros mundos, lo que obliga a Peter a descubrir lo que realmente significa ser Spider-Man.",
            "genero": ["Accion", "Aventura", "Fantasia"],
            "calificacion": 8.2,
            "duracion": 148,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Multiverso", "Doctor Strange", "Enemigos dimensionales", "Identidad secreta", "Responsabilidad heroica", "Amistad", "Poderes mágicos", "Consecuencias del hechizo", "Redención personal", "Aventuras interdimensionales"]
                }
            ]
        },
        {
            "nombre": "Spider-Man: De regreso a casa",
            "ano": 2017,
            "descripcion": "Peter Parker equilibra su vida como estudiante de secundaria en Queens y como el súper héroe Spider Man, esta vez se encuentra a si mismo contra una nueva amenaza asechando Nueva York.",
            "genero": ["Accion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 7.4,
            "duracion": 133,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                     "Value": ["Hombre Araña", "Peter Parker", "Queens", "Superhéroe", "Nueva York", "Aventura", "Acción", "Ciencia Ficción", "Equilibrio", "Amenaza"]
                }
            ]
        },
        {
            "nombre": "Spider-Man: A Través Del Spider-Verso",
            "ano": 2023,
            "descripcion": "Miles Morales regresa para el próximo capítulo de la saga ganadora de un Oscar, una aventura épica que transportará a Brooklyn a tiempo completo.",
            "genero": ["Animacion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 8.6,
            "duracion": 140,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Hombre Araña", "Miles Morales", "Spider-Verso", "Aventura", "Animación", "Ciencia Ficción", "Brooklyn", "Saga", "Oscar", "Portales", "Hombre con manchas"]
                }
            ]
        },
        {
            "nombre": "Jumanji: El siguiente nivel",
            "ano": 2019,
            "descripcion": "La pandilla ha vuelto pero el juego ha cambiado. Cuando regresen para rescatar a uno de los suyos, los jugadores tendrán que enfrentarse a partes desconocidas, desde desiertos áridos hasta montañas nevadas.",
            "genero": ["Accion", "Aventura", "Comedia"],
            "calificacion": 6.7,
            "duracion": 123,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Jumanji", "Pandilla", "Juego", "Desiertos", "Montañas", "Aventura", "Animales", "Comedia", "Rescate", "Videojuego"]
                }
            ]
        },
        {
            "nombre": "Jumanji: En la selva",
            "ano": 2017,
            "descripcion": "Cuatro adolescentes son absorbidos por un videojuego mágico, y la única forma en que pueden escapar es trabajar juntos para terminar el juego.",
            "genero": ["Accion", "Aventura", "Comedia"],
            "calificacion": 7.0,
            "duracion": 119,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Videojuego mágico", "Adolescentes", "Aventura", "Acción", "Comedia", "Escape", "Trabajo en equipo", "Juego", "Absorbidos", "Fantasía"]
                }
            ]
        },
        {
            "nombre": "Harry Potter y las reliquias de la muerte (parte 1)",
            "ano": 2010,
            "descripcion": "Harry se embarca en una carrera contrarreloj para destruir los Horrocruxes y descubre la existencia de los tres objetos más poderosos del mundo mágico: las reliquias de la muerte.",
            "genero": ["Fantasia", "Aventura", "Familia"],
            "calificacion": 7.7,
            "duracion": 166,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Harry Potter", "Escobas Voladoras", "Reliquias de la muerte", "Magia", "Aventura", "Familia", "Carrera contrarreloj", "Mundo mágico", "Amistad", "Poderosos"]
                }
            ]
        },
        {
            "nombre": "Harry Potter y las reliquias de la muerte (parte 2)",
            "ano": 2011,
            "descripcion": "Harry, Ron y Hermione buscan los Horcruxes restantes de Voldemort en su esfuerzo por destruir al Señor Oscuro mientras la batalla final continúa en Hogwarts.",
            "genero": ["Fantasia", "Aventura", "Familia"],
            "calificacion": 8.1,
            "duracion": 130,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Harry Potter", "Ron", "Hermione", "Varitas Magicas", "Voldemort", "Señor Oscuro", "Hogwarts", "Batalla final", "Magia", "Escuela de mago"]
                }
            ]
        },
        {
            "nombre": "El Hobbit: La batalla de los cinco ejércitos",
            "ano": 2014,
            "descripcion": "Bilbo y la compañía se ven forzados a implicarse en una guerra con un abanico de combatientes para mantener la Montaña Solitaria fuera de las manos de la creciente oscuridad.",
            "genero": ["Aventura", "Fantasia"],
            "calificacion": 7.4,
            "duracion": 144,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Bilbo", "Compañía", "Guerra", "Montaña Solitaria", "Oscuridad", "Enanos", "Elfos", "Batalla", "Ejércitos", "Combate", "Ogros"]
                }
            ]
        },
        {
            "nombre": "Guasón",
            "ano": 2019,
            "descripcion": "En Ciudad Gótica, Arthur Fleck, un comediante con problemas mentales, marginado y maltratado por la sociedad. Esto lo lleva a descender a un espiral de revolución, sangre y crimen donde conoce a su alter-ego: El Joker.",
            "genero": ["Crimen", "Drama", "Suspenso"],
            "calificacion": 8.4,
            "duracion": 122,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Ciudad Gótica", "Arthur Fleck", "Comediante", "Problemas mentales", "Revolución", "Crimen", "Payaso", "Suspenso", "Sociedad", "Alter-ego"]
                }
            ]
        },
        {
            "nombre": "El rey león",
            "ano": 2019,
            "descripcion": "Después de la muerte de su padre, un joven león huye de casa para descubrir el verdadero significado de valentía y responsabilidad.",
            "genero": ["Aventura", "Animacion", "Drama"],
            "calificacion": 6.8,
            "duracion": 118,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["León", "Muerte", "Valentía", "Responsabilidad", "Huida", "Aventura", "Leon mata a su hermano", "Scark", "Familia", "Lecciones de vida"]
                }
            ]
        },
        {
            "nombre": "El Escuadrón Suicida",
            "ano": 2021,
            "descripcion": "Los villanos Harley Quinn, Bloodsport, Peacemaker y una colección de estafadores en la prisión de Belle Reve se unen al grupo de trabajo súper secreto y súper sombrío X, en la lejana y abarrotada de enemigos de la isla de Corto Maltese.",
            "genero": ["Aventura", "Accion", "Comedia"],
            "calificacion": 7.2,
            "duracion": 132,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Harley Quinn", "Bloodsport", "Peacemaker", "Villanos", "Prisión Belle Reve", "Grupo de trabajo", "Acción", "Aventura", "Comedia", "Novia del joker"]
                }
            ]
        },
        {
            "nombre": "John Wick 2: Un nuevo día para matar",
            "ano": 2017,
            "descripcion": "Después de la trágica muerte de su esposa, y de su perro, el temido y respetado asesino a sueldo John Wick (Keanu Reeves), se ve obligado a responder a una deuda pendiente con la organización secreta que lo ha cobijado durante años.",
            "genero": ["Accion", "Crimen", "Suspenso"],
            "calificacion": 7.4,
            "duracion": 122,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["John Wick", "Keanu Reeves", "Asesino a sueldo", "Trágica muerte", "Deuda pendiente", "Organización secreta", "Acción", "Crimen", "Suspenso", "Venganza"]
                }
            ]
        },
        {
            "nombre": "Yo antes de ti",
            "ano": 2016,
            "descripcion": "Una joven forma un vínculo improbable con un hombre discapacitado del que está cuidando.",
            "genero": ["Drama", "Romance"],
            "calificacion": 7.4,
            "duracion": 110,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Joven", "Vínculo", "Improbable", "Discapacitado", "Cuidado", "Drama", "Romance", "Emoción", "Relación", "Motivación", "Regalo Medias", "Enamorados"]
                }
            ]
        },
        {
            "nombre": "Los increíbles 2",
            "ano": 2018,
            "descripcion": "Los Incredibles asumen una nueva misión, que implica un cambio en los roles familiares, el Señor Increíble debe encargarse de la casa mientras que su esposa Helen sale a salvar el mundo.",
            "genero": ["Animacion", "Accion", "Aventura"],
            "calificacion": 7.5,
            "duracion": 118,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Increíbles", "Familia", "Roles", "Misión", "Helen", "Señor Increíble", "Salvar el mundo", "Bebe con muchos poderes", "Acción", "Aventura"]
                }
            ]
        },
        {
            "nombre": "Venom",
            "ano": 2018,
            "descripcion": "Un fracasado periodista crea un lazo con una criatura alienígena y decide proteger la Tierra.",
            "genero": ["Accion", "Aventura", "Ciencia Ficcion"],
            "calificacion": 6.6,
            "duracion": 112,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Venom", "Periodista", "Criatura alienígena", "Proteger", "Tierra", "Acción", "Aventura", "Ciencia Ficción", "Antihéroe", "Simbionte"]
                }
            ]
        },
        {
            "nombre": "Aladdin",
            "ano": 2019,
            "descripcion": "Un joven callejero de buen corazón y un Visir hambriento de poder, luchan por una lámpara mágica que tiene el poder de hacer realidad los deseos más profundos.",
            "genero": ["Aventura", "Comedia", "Familia"],
            "calificacion": 6.9,
            "duracion": 128,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Aladdin", "Callejero", "Visir", "Lámpara mágica", "Deseos", "Aventura", "Comedia", "Familia", "Magia", "Fantasía"]
                }
            ]
        },
        {
            "nombre": "El Libro de la Selva",
            "ano": 2016,
            "descripcion": "Mowgli, un niño criado en la selva por una manada de lobos, emprende un fascinante viaje de autodescubrimiento cuando se ve obligado a abandonar el único hogar que ha conocido en toda su vida.",
            "genero": ["Accion", "Aventura"],
            "calificacion": 7.4,
            "duracion": 106,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Mowgli", "Selva", "Lobos", "Autodescubrimiento", "Hogar", "Aventura", "Niño Mono", "Infancia", "Crecimiento", "Animales"]
                }
            ]
        },
        {
            "nombre": "La Bella y la Bestia",
            "ano": 2017,
            "descripcion": "Un príncipe egoísta es maldecido para convertirse en un monstruo por el resto de su vida, a menos que aprenda a enamorarse de una hermosa joven a la que mantiene prisionera.",
            "genero": ["Aventura", "Familia", "Fantasia"],
            "calificacion": 7.1,
            "duracion": 129,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Bella", "Bestia", "Príncipe", "Maldición", "Amor", "Aventura", "Familia", "Fantasía", "Prisionera", "Enseñanza", "Enamorados"]
                }
            ]
        },
        {
            "nombre": "X-Men: Días del futuro pasado",
            "ano": 2014,
            "descripcion": "Los X-men envían a Wolverine al pasado en un desesperado esfuerzo por cambiar la historia y prevenir el fin tanto para los humanos como para los mutantes.",
            "genero": ["Aventura", "Accion", "Aventura"],
            "calificacion": 7.9,
            "duracion": 132,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["X-Men", "Días del futuro pasado", "Wolverine", "Mutantes", "Viaje en el tiempo", "Centinela", "Cambia forma", "Ciencia Ficción", "Desesperación", "Historia"]
                }
            ]
        },
        {
            "nombre": "Star Wars: El despertar de la fuerza",
            "ano": 2015,
            "descripcion": "Tres décadas después de la derrota del Imperio, surge una nueva amenaza en la Primera Orden militante. El soldado de asalto Finn y Rey están atrapados en la búsqueda de la Resistencia por el desaparecido Luke Skywalker.",
            "genero": ["Aventura", "Accion", "Ciencia Ficcion"],
            "calificacion": 7.8,
            "duracion": 148,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Star Wars", "El despertar de la fuerza", "Imperio", "Primera Orden", "Finn", "Rey", "Luke Skywalker", "Naves Espaciales", "Guerra en el espacio", "Ciencia Ficción"]
                }
            ]
        },
        {
            "nombre": "Rápidos y furiosos 9",
            "ano": 2021,
            "descripcion": "Dom y su equipo deben enfrentarse a un terrorista internacional que resulta ser el hermano separado de Dom y Mia.",
            "genero": ["Crimen", "Accion", "Suspenso"],
            "calificacion": 5.2,
            "duracion": 143,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Dom", "Familia", "Carreras", "Crimen", "Terrorista", "Internacional", "Hermano", "Rápido", "Furioso", "Autos"]
                }
            ]
        },
        {
            "nombre": "Los juegos del hambre",
            "ano": 2012,
            "descripcion": "Katniss Everdeen toma voluntariamente el lugar de su hermana menor en los Juegos del Hambre: una competencia televisada en la que dos adolescentes de cada uno de los doce distritos de Panem son elegidos al azar para luchar hasta la muerte.",
            "genero": ["Aventura", "Accion", "Ciencia Ficcion"],
            "calificacion": 7.2,
            "duracion": 142,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Katniss Everdeen", "Juegos del Hambre", "Distritos de Panem", "Competencia televisada", "Aventura", "Acción", "Ciencia Ficción", "Voluntariado", "Lucha", "Sobrevivencia", "Arco en fuego"]
                }
            ]
        },
        {
            "nombre": "Pantera Negra: Wakanda Por Siempre",
            "ano": 2022,
            "descripcion": "El pueblo de Wakanda lucha para proteger su hogar de las potencias mundiales interventoras mientras lloran la muerte de su rey T'Challa.",
            "genero": ["Aventura", "Accion", "Drama"],
            "calificacion": 6.7,
            "duracion": 161,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Wakanda", "Rey T'Challa", "Protección", "Intervención", "Lucha", "Pueblo", "Muerte", "Pantera", "Acción", "Drama"]
                }
            ]
        },
        {
            "nombre": "Animales fantásticos y dónde encontrarlos",
            "ano": 2016,
            "descripcion": "Las aventuras del escritor Newt Scamander en la comunidad secreta de brujas y magos de Nueva York setenta años antes de que Harry Potter lea en la escuela el libro de su autoría.",
            "genero": ["Aventura", "Familia", "Fantasia"],
            "calificacion": 7.2,
            "duracion": 152,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Newt Scamander", "Brujas", "Magos", "Nueva York", "Aventura", "Familia", "Fantasía", "Criaturas mágicas", "Secretos", "Descubrimientos"]
                }
            ]
        },
        {
            "nombre": "Wonka",
            "ano": 2023,
            "descripcion": "La historia se centrará específicamente en un joven Willy Wonka y en cómo conoció a los Oompa-Loompas en una de sus primeras aventuras.",
            "genero": ["Aventura", "Comedia", "Familia"],
            "calificacion": 7.0,
            "duracion": 116,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Willy Wonka", "Oompa-Loompas", "Aventura", "Comedia", "Familia", "Orígenes", "Fantasía", "Chocolate", "Dulces", "Aprendizaje"]
                }
            ]
        },
        {
            "nombre": "Thor: Un mundo oscuro",
            "ano": 2013,
            "descripcion": "Cuando la Dra. Jane Foster es maldecida con una poderosa entidad, Thor tendrá que emprender un viaje muy peligroso para salvar el mundo.",
            "genero": ["Aventura", "Accion", "Fantasia"],
            "calificacion": 6.8,
            "duracion": 112,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Thor", "Jane Foster", "Aventura", "Acción", "Fantasía", "Maldición", "Poderes", "Viaje peligroso", "Salvación", "Héroes"]
                }
            ]
        },
        {
            "nombre": "El regreso de la nana mágica",
            "ano": 2010,
            "descripcion": "La niñera mágica McPhee acude a ayudar a una madre joven y agobiada que intenta administrar la granja familiar mientras su marido está en la guerra.",
            "genero": ["Comedia", "Familia", "Fantasia"],
            "calificacion": 6.1,
            "duracion": 99,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Niñera mágica", "Familia", "Comedia", "Fantasía", "Ayuda mágica", "Granja familiar", "Guerra", "Infantil", "Magia", "Aventuras"]
                }
            ]
        },
        {
            "nombre": "Mundo Jurásico",
            "ano": 2015,
            "descripcion": "En un nuevo parque temático, construido en el sitio original de Jurassic Park, se crea un dinosaurio híbrido genéticamente modificado, el Indominus Rex, que escapa de la contención y se lanza a una ola de matanzas.",
            "genero": ["Aventura", "Accion", "Ciencia Ficcion"],
            "calificacion": 6.9,
            "duracion": 161,
            "metadata": [
                {
                    "Tag": "palabras_claves",
                    "Value": ["Parque temático", "Dinosaurios", "Indominus Rex", "Aventura", "Acción", "Ciencia Ficción", "Genética", "Escape", "Matanza", "Emoción"]
                }
            ]
        },
    ]
}

# Convertir a DataFrame
df = pd.json_normalize(data['movies'])

print(df)

with open('movies.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("Archivo JSON creado exitosamente.")

                                               nombre   ano  \
0                                         Oppenheimer  2023   
1                                            Deadpool  2016   
2                                                Duna  2021   
3                                   Godzilla vs. Kong  2021   
4                                            Godzilla  2014   
5                                     Kung Fu Panda 3  2016   
6                                            It (Eso)  2017   
7                                             Furiosa  2024   
8                              Hasta el último hombre  2016   
9                                           Megalodon  2018   
10                                Wolverine: Inmortal  2013   
11                                    Intensa Mente 2  2024   
12                                      Intensa Mente  2015   
13                               Un lugar en silencio  2018   
14                           Bad Boys Hasta La Muerte  

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.7 MB/s et